In [1]:
# Imports

# Standards
import numpy as np
import pandas as pd

# API
import requests

# Automating
import time
import datetime as dt
import warnings
import sys

**Test Pull from World of Warcraft Subreddit**

In [8]:
# Set base url to get comments
# url = 'https://api.pushshift.io/reddit/search/submission'
url = 'https://api.pushshift.io/reddit/search/comment'

In [9]:
# Set parameters 
params = {
    'subreddit': 'tolkienfans',
    'size': 50,
    'lang': True,   # only the ones that contain usable text data
    'before': 1616977706#, # epoch (another version of a timestampe) 
#     'after': 1616977706
}

In [10]:
# Make request
res = requests.get(url, params)
res

<Response [200]>

In [11]:
# Examine object: as a text? (ugly) as a json? (pretty)

# res.text
res.json()

{'data': [{'all_awardings': [],
   'associated_award': None,
   'author': 'projekt6',
   'author_flair_background_color': None,
   'author_flair_css_class': None,
   'author_flair_richtext': [],
   'author_flair_template_id': None,
   'author_flair_text': None,
   'author_flair_text_color': None,
   'author_flair_type': 'text',
   'author_fullname': 't2_fuij2',
   'author_patreon_flair': False,
   'author_premium': False,
   'awarders': [],
   'body': 'Do you by chance know if there is an I’ll ISA tree slipcase set that matches the recently release Silm?',
   'collapsed_because_crowd_control': None,
   'comment_type': None,
   'created_utc': 1616977560,
   'gildings': {},
   'id': 'gsmwrii',
   'is_submitter': False,
   'link_id': 't3_mfbfyj',
   'locked': False,
   'no_follow': True,
   'parent_id': 't1_gsmolm3',
   'permalink': '/r/tolkienfans/comments/mfbfyj/id_like_to_purchase_a_matching_set_of_books_of/gsmwrii/',
   'retrieved_on': 1616977571,
   'score': 1,
   'send_replies': Tru

**Can I make this cleaner? Can I convert this to a DataFrame?**

In [13]:
tolkien_data = res.json()['data']
df = pd.DataFrame(tolkien_data)
df.head()

,all_awardings,associated_award,author,author_flair_background_color,author_flair_css_class,author_flair_richtext,author_flair_template_id,author_flair_text,author_flair_text_color,author_flair_type,...,score,send_replies,stickied,subreddit,subreddit_id,top_awarded_type,total_awards_received,treatment_tags,author_cakeday,distinguished
0,[],None,projekt6,None,None,[],None,None,None,text,...,1,True,False,tolkienfans,t5_2r6jm,None,0,[],NaN,NaN
1,[],None,Advanced-Fan1272,None,None,[],None,None,None,text,...,1,True,False,tolkienfans,t5_2r6jm,None,0,[],NaN,NaN
2,[],None,HeidiFree,None,None,[],None,None,None,text,...,1,True,False,tolkienfans,t5_2r6jm,None,0,[],NaN,NaN
3,[],None,Advanced-Fan1272,None,None,[],None,None,None,text,...,1,True,False,tolkienfans,t5_2r6jm,None,0,[],NaN,NaN
4,[],None,PhendranaDrifter,None,None,[],None,None,None,text,...,1,True,False,tolkienfans,t5_2r6jm,None,0,[],NaN,NaN


**So many columns - surely we don't need that many, let's drop some**

In [14]:
# Print all the columns and cross-reference with document to identify columns that can be useful in webscraping
for col in df.columns:
    print(col)

all_awardings
associated_award
author
author_flair_background_color
author_flair_css_class
author_flair_richtext
author_flair_template_id
author_flair_text
author_flair_text_color
author_flair_type
author_fullname
author_patreon_flair
author_premium
awarders
body
collapsed_because_crowd_control
comment_type
created_utc
gildings
id
is_submitter
link_id
locked
no_follow
parent_id
permalink
retrieved_on
score
send_replies
stickied
subreddit
subreddit_id
top_awarded_type
total_awards_received
treatment_tags
author_cakeday
distinguished


**Which columns would you want to keep in NLP Project**

In [15]:
df[['subreddit', 'created_utc', 'body', 'author', 'permalink']].head()

,subreddit,created_utc,body,author,permalink
0,tolkienfans,1616977560,Do you by chance know if there is an I’ll ISA ...,projekt6,/r/tolkienfans/comments/mfbfyj/id_like_to_purc...
1,tolkienfans,1616977084,"Eru is intervening all the time. The point is,...",Advanced-Fan1272,/r/tolkienfans/comments/mf7jsu/the_possible_we...
2,tolkienfans,1616976323,Might be cliche...but my favorite has always b...,HeidiFree,/r/tolkienfans/comments/mf3he7/lines_in_tolkie...
3,tolkienfans,1616975938,He's not teasing her. He basically tells her t...,Advanced-Fan1272,/r/tolkienfans/comments/mfaae8/when_faramir_pr...
4,tolkienfans,1616975871,"I guess, from reading the books after seeing t...",PhendranaDrifter,/r/tolkienfans/comments/mfaae8/when_faramir_pr...


In [16]:
tolkien_data[0]

{'all_awardings': [],
 'associated_award': None,
 'author': 'projekt6',
 'author_flair_background_color': None,
 'author_flair_css_class': None,
 'author_flair_richtext': [],
 'author_flair_template_id': None,
 'author_flair_text': None,
 'author_flair_text_color': None,
 'author_flair_type': 'text',
 'author_fullname': 't2_fuij2',
 'author_patreon_flair': False,
 'author_premium': False,
 'awarders': [],
 'body': 'Do you by chance know if there is an I’ll ISA tree slipcase set that matches the recently release Silm?',
 'collapsed_because_crowd_control': None,
 'comment_type': None,
 'created_utc': 1616977560,
 'gildings': {},
 'id': 'gsmwrii',
 'is_submitter': False,
 'link_id': 't3_mfbfyj',
 'locked': False,
 'no_follow': True,
 'parent_id': 't1_gsmolm3',
 'permalink': '/r/tolkienfans/comments/mfbfyj/id_like_to_purchase_a_matching_set_of_books_of/gsmwrii/',
 'retrieved_on': 1616977571,
 'score': 1,
 'send_replies': True,
 'stickied': False,
 'subreddit': 'tolkienfans',
 'subreddit_id

#### Build the API Function

Here we will scale up the previous process to pull many posts, working iteratively backwards using epoch timestamp.

In [18]:
def get_comments(base_url, subreddit, n_iter, epoch_right_now): # subreddit name and number of times function should run
    
    # instantiate empty list
    df_list = []
    
    # save current epoch, used to iterate in reverse through time
    current_time = epoch_right_now
    
    # set up for loop
    for comment in range(n_iter):
        
        params = {
            'subreddit': subreddit,
            'size': 50,
            'lang': True,   
            'before': current_time
        }
        
        # instantiate get request
        res = requests.get(base_url, params)
        
        # take data from most recent request, store as df
        df = pd.DataFrame(res.json()['data'])
        
        # pull specific columns from dataframe for analysis
        df = df[['subreddit', 'created_utc', 'body', 'author', 'permalink']]
        
        # append to empty dataframe list
        df_list.append(df)
        
        # set current time counter back to last epoch in recently grabbed df
        current_time = df['created_utc'].min()   # Gets the oldest post that I grabbed in this for loop iteration
        
        # add wait time
        time.sleep(5)
        
    # return one dataframe for all requests
    return pd.concat(df_list)

In [24]:
reddit_url = url = 'https://api.pushshift.io/reddit/search/comment' 
tolk_subreddit = 'tolkienfans'
tolkien_comments = get_comments(reddit_url, tolk_subreddit, n_iter=5, epoch_right_now=1616998184)

In [25]:
tolkien_comments.head()

,subreddit,created_utc,body,author,permalink
0,tolkienfans,1616997631,Blessings to all of you who have made me cry a...,WaifuYahu,/r/tolkienfans/comments/mf3he7/lines_in_tolkie...
1,tolkienfans,1616997318,This was part of my father's funeral. It's jus...,acousticonion,/r/tolkienfans/comments/mf3he7/lines_in_tolkie...
2,tolkienfans,1616996888,"In January 2020 I went to the Carribbean, made...",CaptainBlagbird,/r/tolkienfans/comments/mf3he7/lines_in_tolkie...
3,tolkienfans,1616996697,"Weird, because I was just thinking about them ...",WaifuYahu,/r/tolkienfans/comments/mf3he7/lines_in_tolkie...
4,tolkienfans,1616995857,We probably also shouldn't forget that the Ste...,preserved_fish,/r/tolkienfans/comments/mfaae8/when_faramir_pr...


In [26]:
tolkien_comments.shape

(250, 5)

In [27]:
reddit_url = url = 'https://api.pushshift.io/reddit/search/comment'
hp_subreddit = 'harrypotter'
hp_comments = get_comments(reddit_url, hp_subreddit, n_iter=5, epoch_right_now=1616998184)

In [28]:
hp_comments.head()

,subreddit,created_utc,body,author,permalink
0,harrypotter,1616998173,Goblet of fire is my favorite although by what...,sunflowergirl96,/r/harrypotter/comments/mfiyj8/does_anyone_fee...
1,harrypotter,1616998170,"Im the opposite, mainly because im not fond of...",ToyVaren,/r/harrypotter/comments/mfiyj8/does_anyone_fee...
2,harrypotter,1616998169,**I am not selling this yet but the design is ...,wassupwthecyphe,/r/harrypotter/comments/mfj0k4/would_anyone_be...
3,harrypotter,1616998067,"Naw I thought they are all spot on, no complai...",42yearoldorphan,/r/harrypotter/comments/mfiyj8/does_anyone_fee...
4,harrypotter,1616998007,I found the non-Harry chapters enjoyable and I...,metametatron4,/r/harrypotter/comments/mepnlh/i_would_have_lo...


In [29]:
hp_comments.shape

(250, 5)

**Combine the DataFrames**

In [36]:
posts = pd.concat([tolkien_comments, hp_comments])
posts.head()

,subreddit,title,created_utc,selftext,author,media_only,permalink
0,tolkienfans,What happened to Tolkien’s Red Book?,1616977706,I was wondering if somebody could clear up my ...,lukeskinwalker69epic,False,/r/tolkienfans/comments/mfdrms/what_happened_t...
1,tolkienfans,What Makes People Turn Invisible When the Put ...,1616971688,I guess this is a big question because I never...,Jpmaniax26,False,/r/tolkienfans/comments/mfbzjh/what_makes_peop...
2,tolkienfans,"Theory: At some point not told in the Tale, Ar...",1616971077,This is the last dialogue between Queen Undómi...,yew_eater,False,/r/tolkienfans/comments/mfbsks/theory_at_some_...
3,tolkienfans,I’d like to purchase a matching set of books o...,1616969977,Can anyone help me out? Looking for matching b...,GoosePants72,False,/r/tolkienfans/comments/mfbfyj/id_like_to_purc...
4,tolkienfans,"When Faramir ‘proposes’ to Eowyn, something we...",1616966528,After he confesses his love for her and talks ...,PhendranaDrifter,False,/r/tolkienfans/comments/mfaae8/when_faramir_pr...


In [34]:
comments['subreddit'].value_counts()

tolkienfans    500
harrypotter    500
Name: subreddit, dtype: int64

In [37]:
comments.to_csv('../../data/reddit_comments.csv')

#### Create and run a script

1. Add in time breaks between loops to not overload the API
2. Convert to .py
3. Utilize `caffeinate` on terminal